In [1]:
import time
import feeder
from koinex import Koinex
from bitbns import Bitbns
from binance_site import Binance
from api.arbitrage import withdrawal_fee

In [2]:
#init all sites
k=Koinex()
b=Bitbns()
bnn=Binance()

In [3]:
sites={
    "koinex":k,
    "bitbns":b,
    "binance":bnn
}

In [4]:
def do_arb():
    k.update_balances()
    b.update_balances()
    balance_list={
    "koinex":1*k.wallets["inr"],
    "bitbns":1*b.wallets["inr"]
    }
    arb_list=feeder.feed(balance_list)
    print(arb_list[:1])
    if len(arb_list):
        arb=arb_list[0]
        buyIndian,inter,sellIndian=arb[0].split("_")
        coin1,coin2=arb[1].split("_")
        coin1_initBal=sites[buyIndian.lower()].wallets[coin1]
        coin2_initBal=sites[sellIndian.lower()].wallets[coin2]
        coin1_withdrawal_address=sites[inter.lower()].deposit_credentials(coin1)
        coin2_withdrawal_address=sites[sellIndian.lower()].deposit_credentials(coin2)
    
        new_details=arb[2][1]
        amount=new_details["amount"]*0.9
        indian_buy_details=new_details[coin1]
        indian_sell_details=new_details[coin2]
        coin1_buy_vol,coin1_buy_amnt,coin1_buy_q_price,coin1_buy_avg_price=sites[buyIndian.lower()].decide_buy_details(coin1,amount,indian_buy_details["quote_price"])
        coin2_sell_calc_average,_=sites[sellIndian.lower()].sell_limit_quote(coin2,0.9*indian_sell_details["vol"])
        
        if sites[buyIndian.lower()].feasible_buy(coin1_buy_amnt,amount) and \
        sites[buyIndian.lower()].check_withdrawal_working(coin1) and \
        sites[sellIndian.lower()].check_deposit_working(coin2) and (
            sites[inter.lower()].deposit_credentials(coin1) and \
        sites[inter.lower()].withdraw_enable(coin2) and sites[inter.lower()].deposit_enable(coin1))and (
              sites[sellIndian.lower()].feasible_sell(coin2_sell_calc_average,indian_sell_details["avg price"])):
            
            sites[buyIndian.lower()].place_buy_order(coin1,coin1_buy_q_price,coin1_buy_vol)
            time.sleep(3)
            if sites[buyIndian.lower()].are_open_orders():
                sites[buyIndian.lower()].cancel_all_orders(coin1)
            coin1_bal=sites[buyIndian.lower()].wallets[coin1]-coin1_initBal
            bought_amnt=coin1_bal*coin1_buy_avg_price
            if coin1_bal<0.5*coin1_buy_vol:
                coin1_sell_quote_price=sites[buyIndian.lower()].sell_market_price(coin1)
                sites[buyIndian.lower()].place_sell_order(coin1,coin1_sell_quote_price,coin1_bal)
                return 
            print("bought"+coin1+"@"+buyIndian+":"+str(coin1_bal))    
            sites[buyIndian.lower()].withdraw_crypto(coin1,coin1_bal,coin1_withdrawal_address["address"],coin1_withdrawal_address["addressTag"])
            time.sleep(200)
            
            coin1_withdrew_vol=coin1_bal-withdrawal_fee[buyIndian.lower()][coin1]
            print("withdrew"+coin1+"@"+buyIndian+":"+str(coin1_withdrew_vol))
            while(float(sites[inter.lower()].balance(coin1)["free"])<0.6*coin1_withdrew_vol):
                time.sleep(60)
            print("deposited"+coin1)
            sites[inter.lower()].place_sell_order(coin1)
            print("sold"+coin1+"@"+inter)
            time.sleep(2)
            coin2_int_init_bal=float(sites[inter.lower()].balance(coin2)["free"])
            sites[inter.lower()].place_buy_order(coin2)
            print("bought"+coin2+"@"+inter)
            time.sleep(5)
            coin2_bal=float(sites[inter.lower()].balance(coin2)["free"])
            time.sleep(5)
            sites[inter.lower()].withdraw_crypto(coin2,coin2_bal,coin2_withdrawal_address["address"],coin2_withdrawal_address["tag"])
            coin2_withdrew_vol=coin2_bal-withdrawal_fee[inter.lower()][coin2]
            print("withdrew"+coin2+"@"+inter+":"+str(coin2_bal))
            
            time.sleep(200)
            sites[sellIndian.lower()].market_page(coin2)
            while(float(sites[sellIndian.lower()].wallets[coin2])-coin2_initBal<0.5*coin2_withdrew_vol):
                time.sleep(60)
                sites[sellIndian.lower()].update_cryptobalances(coin2)
            print("deposited"+coin2+"@"+sellIndian)
            coin2_ind_bal=sites[sellIndian.lower()].wallets[coin2]-coin2_initBal
            coin2_ind_sell_avg,coin2_quote_price=sites[sellIndian.lower()].sell_limit_quote(coin2,coin2_ind_bal)
            if coin2_ind_sell_avg<(bought_amnt/coin2_ind_bal) and not(coin2 in ["neo","qtum"]):
                coin2_quote_price=1.002*(bought_amnt/coin2_ind_bal)
                if coin2_quote_price>indian_sell_details["quote_price"]:
                    coin2_quote_price=indian_sell_details["quote_price"]
            if coin2 in ["neo","qtum"]:
                if coin2_ind_sell_avg<(bought_amnt/(coin2_bal-coin2_int_init_bal)):
                    coin2_quote_price=1.002*(bought_amnt/(coin2_bal-coin2_int_init_bal))
                    if coin2_quote_price>indian_sell_details["quote_price"]:
                        coin2_quote_price=indian_sell_details["quote_price"]
            sites[sellIndian.lower()].place_sell_order(coin2,coin2_quote_price,coin2_ind_bal)
            print("placed sell order:"+str(coin2_quote_price))
            print("\n")

In [5]:
def close_allsites(indian_sites_list):
    try:
        for each in indian_sites_list:
            each.signout()
    finally:
        for each in indian_sites_list:
            each.close()

In [7]:
from threading import Thread
def all_indiansites_login(indian_sites_list):
    threads=[]
    for each in indian_sites_list:
        t=Thread(target=each.signin)
        threads.append(t)
        t.start()
    for thread in threads:
        thread.join()
        

In [9]:
def main():
    try:
        all_indiansites_login([k,b])
        login_time=time.time()
        while(time.time()-login_time<3600*4):
            try:
                do_arb()
            except Exception as e:
                print(e)
    finally:
        close_allsites([k,b])

In [12]:
while(True):
    main()

Message: 

popup not present
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[('Bitbns_Binance_Bitbns', 'ada_ven', (({'ada': {'arb': 1.022582495856261, 'buy': 12.2, 'sell': 2.543e-05}, 'ven': {'arb': 1.0329709960961193, 'buy': 0.00025015, 'sell': 126.1}}, 1.0562980593350977), {'ada': {'new_arb': 2.0844262295081965e-06, 'quote_price': 12.2, 'avg price': 12.2, 'vol': 646.2893442622951, 'amnt': 7884.73}, 'ven': {'new_arb': 503457.92524485313, 'quote_price': 125.0, 'avg price': 125.94, 'vol': 62.60912799999999, 'amnt': 7884.73}, 'amount': 7884.73}, 1.027129799724896))]
[]
[('Bitbns_Binance_Bitbns', 'zrx_ven', (({'zrx': {'arb': 1.0584948847578985, 'buy': 78.79, 'sell': 0.00017}, 'ven': {'arb': 1.0248117888387265, 'buy': 0.00025075, 'sell': 126.1}}, 1.0847580363253837), {'zrx': {'new_arb': 2.157634217540297e-06, 'quote_price': 78.79, 'avg price': 78.79, 'vol': 100.07272496509708, 'amnt': 7884.73}, 'ven': {'new_arb': 502253.24027916254, 'quote_price': 125.0, 'avg price': 125.94, 'vol': 62.609127

Exception in thread Thread-70:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/chennam/python scripts/arbitrage_scripts/api/arbitrage.py", line 232, in get_orderbooks_for_list_of_coins
    orderbook_dict[coin]=orderbook_mod.orderbook(coin)
  File "/home/chennam/python scripts/arbitrage_scripts/orderbook/koinex.py", line 122, in orderbook
    buy_orders=self.buy_orderbook()
  File "/home/chennam/python scripts/arbitrage_scripts/orderbook/koinex.py", line 110, in buy_orderbook
    order=self.find_column_text(row)
  File "/home/chennam/python scripts/arbitrage_scripts/orderbook/koinex.py", line 69, in find_column_text
    columns=row.find_elements_by_tag_name("td")
  File "/home/chennam/.local/lib/python3.6/site-packages/selenium/webdriver/remote/webelement.py", line 320, in find_elements_by_tag_nam

Traceback (most recent call last):
  File "<ipython-input-9-6f1195257bde>", line 7, in main
    do_arb()
  File "<ipython-input-4-a8d66b757970>", line 8, in do_arb
    arb_list=feeder.feed(balance_list)
  File "/home/chennam/python scripts/arbitrage_scripts/feeder.py", line 46, in feed
    indian_withdrawal_exclusion_names)
  File "/home/chennam/python scripts/arbitrage_scripts/api/arbitrage.py", line 314, in main
    return approved_triangular_arbitrages(refined_triangular_arb,balance_list,orderbooks)
  File "/home/chennam/python scripts/arbitrage_scripts/api/arbitrage.py", line 252, in approved_triangular_arbitrages
    list_of_orderbooks=get_site_orderbooks(orderbooks,list_of_coins)
  File "/home/chennam/python scripts/arbitrage_scripts/api/arbitrage.py", line 247, in get_site_orderbooks
    thread.join()
  File "/usr/lib/python3.6/threading.py", line 1056, in join
    self._wait_for_tstate_lock()
  File "/usr/lib/python3.6/threading.py", line 1072, in _wait_for_tstate_lock
    elif

TypeError: must be str, not list